<h1 style="text-align: center;" markdown="1">TP2: Random Forest sin ningun tuneo de parametros</h1>
![](../data/icon_properati-data.png)

In [1]:
from sklearn import svm
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV

import pandas as pd

# modules
import knn as knnlibrary

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#leo dataset
properties =knnlibrary.get_dataset()

# transformo el campo fecha
properties_caba = knnlibrary.transform_date(properties)

# filtro por CABA y GBA
# queremos solo las propiedades que tienen precio y eliminamos columnas que sabemos que no son 
#redundantes y que no nos servirian para knn
# eliminamos propiedades con mas de 54 pisos
properties_caba = knnlibrary.clean_dataset(properties)

# las expensas tienen demasiados nulos por lo que voy a eliminar esa columna
properties_caba = properties_caba.drop(['expenses'], axis = 1)

# atributos categoricos
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
properties_caba = knnlibrary.encoder_attributes(properties_caba, encoder)

properties_caba.tail()

# eliminamos filas con valores nulo
properties_caba = properties_caba.dropna(how='any')
properties_caba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10364 entries, 385 to 80073
Data columns (total 14 columns):
property_type              10364 non-null int64
place_name                 10364 non-null int64
place_with_parent_names    10364 non-null int64
state_name                 10364 non-null int64
lat                        10364 non-null float64
lon                        10364 non-null float64
price                      10364 non-null float64
surface_total_in_m2        10364 non-null float64
surface_covered_in_m2      10364 non-null float64
floor                      10364 non-null float64
rooms                      10364 non-null float64
created_on_year            10364 non-null int64
created_on_month           10364 non-null int64
created_on_day             10364 non-null int64
dtypes: float64(7), int64(7)
memory usage: 1.2 MB


/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
import datetime

now = datetime.datetime.now()

In [15]:
from sklearn.model_selection import train_test_split

# separamos el train de traing para validarlo luego usando un 20% de los datos
X, y = properties_caba.iloc[:, properties_caba.columns != 'price'].values, properties_caba.iloc[:, properties_caba.columns == 'price'].values
Xtrn, Xtest, Ytrn, Ytest = train_test_split(X, y, test_size=0.3, random_state=now.microsecond)

model = RandomForestRegressor(n_estimators=1500, max_features='sqrt')
model.fit(Xtrn, Ytrn)

r2_score(Ytest, model.predict(Xtest))

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.79077913240718367

# Prediction

In [4]:
# leemos set de test
test_df = pd.read_csv('../data/test/properati_dataset_testing_noprice.csv', low_memory=False)
test_df.head()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...


In [5]:
# transformamos atributos categoricos
test_df['place_name'] = encoder.fit_transform(test_df[['place_name']])
test_df['state_name'] = encoder.fit_transform(test_df[['state_name']])
test_df['place_with_parent_names'] = encoder.fit_transform(test_df[['place_with_parent_names']])
test_df['property_type'] = encoder.fit_transform(test_df[['property_type']])

# tranformamos fechas
X_test_df = knnlibrary.transform_date(test_df)
X_test_df = X_test_df[['floor', 'lat', 'lon', 'place_name', 'place_with_parent_names',
       'property_type', 'rooms', 'state_name', 'surface_covered_in_m2',
       'surface_total_in_m2', 'created_on_year', 'created_on_month',
       'created_on_day']]

# completamos valores nan
from sklearn.preprocessing import Imputer
imputer_mean = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_test_df['floor'] = X_test_df[['floor']].fillna(1)
X_test_df['rooms'] = X_test_df[['rooms']].fillna(1)

X_test_df["surface_total_in_m2"] = imputer_mean.fit_transform(X_test_df[["surface_total_in_m2"]])
X_test_df["surface_covered_in_m2"] = imputer_mean.fit_transform(X_test_df[["surface_covered_in_m2"]])
X_test_df["lat"] = imputer_mean.fit_transform(X_test_df[["lat"]])
X_test_df["lon"] = imputer_mean.fit_transform(X_test_df[["lon"]])

#X_test_std_df = stdsc.transform(X_test_df)

X_test_df.head()

/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:20:

,floor,lat,lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,created_on_year,created_on_month,created_on_day
0,1.0,-34.610988,-58.363464,152,219,1,1.0,4,359.471588,0.0,2017,8,24
1,1.0,-34.629923,-58.465820,29,180,1,1.0,3,359.471588,0.0,2017,8,25
2,1.0,-34.589363,-58.412880,136,210,1,1.0,4,48.000000,53.0,2017,8,1
3,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1
4,1.0,-34.629923,-58.465820,40,193,1,1.0,4,51.000000,51.0,2017,8,1


In [6]:
# prediccion
X_test_df['prediction'] = model.predict(X_test_df)
X_test_df.head(10)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,floor,lat,lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,created_on_year,created_on_month,created_on_day,prediction
0,1.0,-34.610988,-58.363464,152,219,1,1.0,4,359.471588,0.0,2017,8,24,228160.093267
1,1.0,-34.629923,-58.465820,29,180,1,1.0,3,359.471588,0.0,2017,8,25,224175.770067
2,1.0,-34.589363,-58.412880,136,210,1,1.0,4,48.000000,53.0,2017,8,1,320276.131800
3,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1,320276.131800
4,1.0,-34.629923,-58.465820,40,193,1,1.0,4,51.000000,51.0,2017,8,1,320276.131800
5,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1,320276.131800
6,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1,320276.131800
7,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1,320276.131800
8,1.0,-34.578721,-58.457095,47,195,1,1.0,4,67.000000,67.0,2017,8,1,320276.131800
9,1.0,-34.629923,-58.465820,7,184,1,1.0,4,47.000000,47.0,2017,8,1,320276.131800


In [7]:
output = pd.DataFrame( data={"id":test_df["id"], "price_usd":X_test_df['prediction']} )

In [8]:
# guardamos
output.to_csv( "../data/result/result_randomforest_"+str(now)+".csv", index=False, quoting=3 )